In [3]:
import cv2
import os
import logging
import numpy as np

# Set up logging
logging.basicConfig(level=logging.INFO)

def extract_nuclei(image_path, output_folder, threshold_value=150, output_format='png',
                  lower_violet=np.array([125, 50, 50]), upper_violet=np.array([160, 255, 255]),
                  min_contour_area=100, min_circularity=0.7):
    """
    Extracts nuclei from a histopathology image and saves them as separate images.

    Parameters:
    - image_path: str, path to the input image.
    - output_folder: str, path to the folder where extracted images will be saved.
    - threshold_value: int, threshold value for binary thresholding.
    - output_format: str, format for saving extracted images ('png').
    - lower_violet: np.array, lower HSV range for nucleus color segmentation.
    - upper_violet: np.array, upper HSV range for nucleus color segmentation.
    - min_contour_area: int, minimum area of contours to be considered as nuclei.
    - min_circularity: float, minimum circularity of contours to be considered as nuclei.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path '{image_path}' could not be loaded. Please check the path.")

    # Convert to HSV color space for better color segmentation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a mask for nuclei based on color
    mask = cv2.inRange(hsv, lower_violet, upper_violet)

    # Apply morphological operations to remove small noises and fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image, image, mask=mask)

    # Convert the result to grayscale
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)

    # Blur the image to reduce noise before thresholding
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Adaptive thresholding for better handling of varying lighting conditions
    thresh = cv2.adaptiveThreshold(blurred, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours of the nuclei
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Check if any contours were found
    if not contours:
        logging.info("No contours found in the image.")
        return

    saved_count = 0
    # Extract and save each nucleus
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area < min_contour_area:
            logging.debug(f"Contour {i} skipped due to small area: {area}")
            continue

        # Calculate circularity to ensure the contour is roughly circular
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            logging.debug(f"Contour {i} has zero perimeter and is skipped.")
            continue
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        if circularity < min_circularity:
            logging.debug(f"Contour {i} skipped due to low circularity: {circularity:.2f}")
            continue

        x, y, w, h = cv2.boundingRect(contour)
        nucleus_image = image[y:y+h, x:x+w]

        # Check if the nucleus image is not empty and meets size requirements
        if nucleus_image.size == 0 or w < 10 or h < 10:
            logging.warning(f"Nucleus image {i} is too small or empty and will not be saved.")
            continue

        # Optional: Further validation can be added here (e.g., checking intensity)

        nucleus_image_path = os.path.join(output_folder, f'nucleus_{i}.{output_format}')
        cv2.imwrite(nucleus_image_path, nucleus_image)
        logging.info(f"Saved nucleus image: {nucleus_image_path}")
        saved_count += 1

    logging.info(f"Total nuclei saved: {saved_count}")

# Example usage
if __name__ == "__main__":
    extract_nuclei(
        image_path='./../original-image/img32.tif',
        output_folder='./../extracted-nuclei',
        threshold_value=150,
        output_format='png',
        lower_violet=np.array([125, 50, 50]),
        upper_violet=np.array([160, 255, 255]),
        min_contour_area=100,
        min_circularity=0.7
    )

INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_307.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_316.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_343.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_515.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_520.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_531.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_569.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_646.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_647.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_668.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_679.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_867.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_868.png
INFO:root:Saved nucleus image: ./../extracted-nuclei/nucleus_901.png
INFO:root:Saved nucleus image: ./.